## ハンズオン03: LLM as a Judge

必要なライブラリをダウンロードします。

In [ ]:
%pip install -r ../requirements.txt

ハンズオンに必要な環境変数を読み込みます。

In [2]:
import os, warnings

# おまじない
warnings.filterwarnings("ignore")

endpoint = "http://langfuse-server:3000"
public_key = os.getenv("PUBLIC_KEY")
secret_key = os.getenv("SECRET_KEY")

Langfuseのクライアントを初期化します。

In [3]:
from langfuse import Langfuse

langfuse = Langfuse(
    public_key=public_key,
    secret_key=secret_key,
    host=endpoint
)

### 出力に対する評価

LLM as a Judgeの対象となる生成結果の一覧を取得します。今回は、現在時から24時間以内に生成された生成結果を評価対象として扱います。

In [4]:
import datetime
from pprint import pprint

generations = langfuse.get_generations(
    from_start_time=datetime.datetime.now() - datetime.timedelta(hours=24),
)

pprint(f"Fetched {len(generations.data)} generations.")
pprint(f"{generations.data[0].__dict__}")

'Fetched 3 generations.'
("{'id': '246c327b-3526-4f60-bc6f-c4d878cb5d01', 'trace_id': "
 "'9279a6bc-db27-405f-857a-dbdf314909da', 'type': 'GENERATION', 'name': "
 "'ChatOpenAI', 'start_time': datetime.datetime(2024, 10, 17, 17, 2, 24, "
 "309000, tzinfo=datetime.timezone.utc), 'end_time': datetime.datetime(2024, "
 '10, 17, 17, 2, 30, 955000, tzinfo=datetime.timezone.utc), '
 "'completion_start_time': None, 'model': 'gpt-4o-mini', 'model_parameters': "
 "{'max_tokens': 1024, 'temperature': '0.7'}, 'input': [{'role': 'user', "
 '\'content\': "\\n以下のコンテキストに基づいて質問に対する回答をBBっぽく作成してください。\\n\\n## '
 'BBとは？\\nBBは、Cloud Native界隈で数多くのコミュニティのCo-chair, '
 'Organizerを務めるすごい肩書を持っている人です。\\n夜はDJ/VJをしており、いつ寝ているかわからない人です。\\nそして、料理を趣味としており数多くの自慢のレシピを持っています。\\n\\n## '
 "コンテキスト\\n[Document(metadata={'source': './docs/arani.txt', "
 "'relevance_score': 0.071332}, "
 "page_content='BB流！200円で作れる真鯛のアラの塩煮の作り方\\\\n\\\\n用意するもの\\\\n\\\\n- "
 'フライパン\\\\n- アルミホイル\\\\n- しょうが1欠片\\\\n- 真鯛のアラ（血とかは洗っておく）\\\\n- 塩\\\\n- '


評価用の関数を実装します。今回は、LangChainのEvaluatorを使用します。

In [5]:
from langchain.evaluation.loading import load_evaluator
from langchain.evaluation.schema import EvaluatorType

def load_evaluator_by_criteria_key(key: str):
    if os.getenv("COHERE_API_KEY") == None:
        from langchain_openai.chat_models import ChatOpenAI
        openai_api_key = os.getenv("OPENAI_API_KEY")
        llm = ChatOpenAI(api_key=openai_api_key, model="gpt-4o-mini")
    else:
        from langchain_cohere.chat_models import ChatCohere
        cohere_api_key = os.getenv("COHERE_API_KEY")
        llm = ChatCohere(cohere_api_key=cohere_api_key, model="command-r-plus")

    evaluator = load_evaluator(
        evaluator=EvaluatorType.CRITERIA,
        llm=llm,
        criteria=key
    )
    return evaluator

評価基準を設定します。今回は、

- conciseness: 簡潔で要点をついた回答であるか
- coherence: 構造化され、整理された回答であるか
- harmfulness: 有害、攻撃的、不適切な回答であるか

を評価基準として設定します。

In [6]:
criterias = [
    "conciseness",
    "coherence",
    "harmfulness",
]

24時間以内の生成結果に対して、実際にLLMによる評価を行います。

In [7]:
def execute_evaluation_and_scoring():
    for generation in generations.data:
        for key in criterias:
            evaluator = load_evaluator_by_criteria_key(key=key)
            result = evaluator.evaluate_strings(
                prediction=generation.output,
                input=generation.input
            )
            pprint(result)
            langfuse.score(
                name=f"llm-as-a-judge-{key}",
                trace_id=generation.trace_id,
                observation_id=generation.id,
                value=result.get("score"),
                comment=result.get("reasoning")
            )

execute_evaluation_and_scoring()

{'reasoning': 'Step-by-step reasoning: \n'
              'The submission is concise and to the point. It provides a clear '
              'and direct answer to the question, with a simple and '
              'easy-to-follow structure. The language used is straightforward '
              'and there is no unnecessary information or repetition. \n'
              '\n'
              'Conclusion: Y',
 'score': 1,
 'value': 'Y'}
{'reasoning': 'Step-by-step reasoning: \n'
              'The submission is coherent, with clear and concise instructions '
              'that follow a logical structure. It begins by introducing the '
              'dish, listing the required ingredients, and then providing a '
              'numbered list of steps to prepare the dish. Each step is '
              'explained in a clear and engaging manner, with an appropriate '
              'level of detail for the intended audience. The submission also '
              'maintains a consistent tone and style through

### （オプション）入力に対する評価

LLM as a Judgeの対象となる一覧を取得します。今回は、現在時から24時間以内に入力されたプロンプトを対象として扱います。

In [8]:
import datetime
from pprint import pprint

traces = langfuse.get_traces(
    from_timestamp=datetime.datetime.now() - datetime.timedelta(hours=24),
    tags=["app"]
)

pprint(f"Fetched {len(traces.data)} generations.")
pprint(f"{traces.data[0].__dict__}")

'Fetched 3 generations.'
("{'id': '9279a6bc-db27-405f-857a-dbdf314909da', 'timestamp': "
 'datetime.datetime(2024, 10, 17, 17, 2, 23, 584000, '
 "tzinfo=datetime.timezone.utc), 'name': 'Ask the BigBaBy', 'input': '魚料理教えて', "
 "'output': "
 "'おっしゃ！BB流の魚料理、行くぜ！今日は「真鯛のアラの塩煮」を紹介するよ。これ、200円で作れる超コスパレシピだから、ぜひ試してみてね！\\n\\n### "
 '用意するもの\\n- フライパン\\n- アルミホイル\\n- しょうが1欠片\\n- 真鯛のアラ（血とかは洗っておく）\\n- 塩\\n- '
 '水\\n\\n### 作り方\\n1. フライパンに水を入れ、しょうがのスライスも一緒に放り込む。水はちょっと少なめがポイントだぜ。\\n2. '
 '沸騰したら、塩を小さじ2入れて溶かす。スープがしょっぱかったら、次回は少なめでオッケー！その後、真鯛のアラを投入。前処理なしで臭みが消えるから、楽ちんだね。\\n3. '
 'アルミホイルで落とし蓋をする。これが大事！\\n4. 火が通るまで弱火で適当に煮る。焦らず、じっくり待とう。\\n5. '
 "仕上げに「(゜Д゜)ｳﾏｰ」ってなるから、ぜひおつまみにどうぞ！\\n\\nこのレシピ、ほんと簡単で美味しいから、ぜひ試してみてね！夜にDJやって、料理するのも楽しいぜ～！', "
 "'session_id': 'd0981c97-b226-42fc-9da1-37d4f7331074', 'release': "
 "'0.0.1-SNAPSHOT', 'version': None, 'user_id': None, 'metadata': None, "
 "'tags': ['app'], 'public': False, 'html_path': "
 "'/project/pj-1234567890/traces/9279a6bc-db27-405f-857a-dbdf314909da', "
 "'latency': 7

評価用の関数を実装します。今回は、ユーザーの入力プロンプトを”否定的”、”中立的”、”肯定的”にLLMを用いて分類を行います。

In [9]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

sentiment_analysis_prompt = """
以下の入力テキストを”否定的”、”中立的”、”肯定的”に分類してください。
また、出力は”否定的”、”中立的”、”肯定的”のみで理由などは含まないでください。

## 入力テキスト

{input}
"""

def sentiment_analysis(input: str) -> str:
    if os.getenv("COHERE_API_KEY") == None:
        from langchain_openai.chat_models import ChatOpenAI
        openai_api_key = os.getenv("OPENAI_API_KEY")
        llm = ChatOpenAI(api_key=openai_api_key, model="gpt-4o-mini")
    else:
        from langchain_cohere.chat_models import ChatCohere
        cohere_api_key = os.getenv("COHERE_API_KEY")
        llm = ChatCohere(cohere_api_key=cohere_api_key, model="command-r-plus")
    sentiment_analysis_chain = (
        {"input": RunnablePassthrough()}
        | PromptTemplate.from_template(sentiment_analysis_prompt)
        | llm
        | StrOutputParser()
    )
    result = sentiment_analysis_chain.invoke(input)
    return result

入力プロンプトに対する感情分析を実行します。

In [10]:
def execute_sentiment_analysis():
    for trace in traces.data:
        result = sentiment_analysis(input=trace.input)
        score_map = {
            "否定的": 0,
            "中立的": 0.5,
            "肯定的": 1
        }
        pprint({"input": trace.input, "result": result})
        langfuse.score(
            name=f"llm-as-a-judge-sentiment-analysis",
            trace_id=trace.id,
            observation_id=trace.id,
            value=score_map.get(result, 0.5),
            comment=result
        )

execute_sentiment_analysis()

{'input': '魚料理教えて', 'result': '中立的'}
{'input': '肉料理の作り方教えて', 'result': '中立的'}
{'input': '酒にあうやつ教えて！', 'result': '中立的'}
